# 分布式Python编程

接下来看看PySpark和big data processing。首先，补充一点基本概念。主要参考了：[hadoop和spark的区别和联系](https://zhuanlan.zhihu.com/p/70455833#:~:text=Hadoop%E5%92%8CSpark%E4%B8%A4%E8%80%85,%E5%80%9F%E5%8A%A9hdfs%E7%9A%84%E6%95%B0%E6%8D%AE%E5%AD%98%E5%82%A8%E3%80%82)。

Hadoop是一个由Apache基金会所开发的分布式系统基础架构。 Hadoop实现了一个分布式文件系统HDFS。HDFS有高容错性的特点，并且设计用来部署在低廉的硬件上；而且它提供高吞吐量来访问应用程序的数据，适合那些有着超大数据集的应用程序。Hadoop的框架最核心的设计就是：HDFS和MapReduce。HDFS为海量的数据提供了存储，而MapReduce则为海量的数据提供了计算

Spark 是专为大规模数据处理而设计的快速通用的计算引擎。Spark拥有Hadoop MapReduce所具有的优点，Spark在Job中间输出结果可以保存在内存中，从而不再需要读写HDFS，因此Spark性能以及运算速度高于MapReduce（hadoop的MapReduce是分步对数据进行处理的，从磁盘中读取数据，进行一次处理，将结果写到磁盘，然后在从磁盘中读取更新后的数据，再次进行的处理，最后再将结果存入磁盘，这种存取磁盘的过程会影响处理速度。spark从磁盘中读取数据，把中间数据放到内存中，完成所有必须的分析处理，将结果写回集群，所以spark更快）。

Hadoop和Spark两者都是大数据框架，但是各自应用场景是不同的。**Hadoop是一个分布式数据存储架构**，它将巨大的数据集分派到一个由普通计算机组成的集群中的多个节点进行存储，降低了硬件的成本。Spark是**一个专门用来对那些分布式存储的大数据进行处理的工具**，它要借助hdfs的数据存储。Hadoop提供分布式数据存储功能HDFS，还提供了用于数据处理的MapReduce。 MapReduce是可以不依靠spark数据的处理的。当然spark也可以不依靠HDFS进行运作，它可以依靠其它的分布式文件系统。但是两者完全可以结合在一起，**hadoop提供分布式 集群和分布式 文件系统，spark可以依附在hadoop的HDFS代替MapReduce弥补MapReduce计算能力不足的问题**。总结一句话：spark在hadoop肩膀上可以让大数据跑的更快。

## 简单了解PySpark and Big Data Processing

现在越来越多地会碰到在自己地机器上要处理大数据地情况。幸运的是像Apache Spark, Hadoop，和其他相关技术已经被开发来解决这样的问题。这些系统的强大功能可以通过使用PySpark直接通过Python得到利用。

无论您是数据科学家，Web开发人员还是介于两者之间的任何人员，任何Python开发人员都可以有效地处理千兆字节甚至更多的数据集。

- 哪些Python概念可以应用于大数据
- 如何使用Apache Spark和PySpark
- 如何编写基本的PySpark程序
- 如何在本地小型数据集上运行PySpark程序
- 将PySpark技能带到分布式系统的下一步

在处理大数据时，函数式编程是一种常见的范例。函数式编程可以得到Embarrassingly parallel代码，即方便地并行。这意味着获取代码并使其在多个CPU甚至完全不同的计算机上运行更加容易。通过一次在多个系统上运行，可以解决单个工作站的物理内存和CPU限制。

这是PySpark生态系统的强大功能，可获取函数代码并自动将其分布在整个计算机群集中。

幸运的是，对于Python程序员而言，函数式编程的许多核心思想都可以在Python的标准库和内置函数中找到。无需离开Python就可以学习大数据处理所需的许多概念。

函数式编程的核心思想是，数据应由函数操纵而不保持任何外部状态。这意味着代码避免使用全局变量，并且始终返回新数据，而不是就地操纵数据。

函数式编程中的另一个常见想法是匿名函数。Python使用lambda关键字公开匿名函数。

先看看lambda函数。Python中的lambda函数是内联定义的，并且仅限于单个表达式。lambda使用内置函数时，您可能已经看过sorted()函数：

In [3]:
x = ['Python', 'programming', 'is', 'awesome!']
print(sorted(x))
['Python', 'awesome!', 'is', 'programming']
print(sorted(x, key=lambda arg: arg.lower()))

['Python', 'awesome!', 'is', 'programming']
['awesome!', 'is', 'programming', 'Python']


内置的filter()，map()和reduce()功能在函数式编程所有常见，这些概念是构成PySpark程序功能的重要部分，了解这些功能很重要。然后，将该知识转换为PySpark程序和Spark API。

filter()根据条件（通常表示为lambda函数）从可迭代项中过滤掉项目：

In [4]:
x = ['Python', 'programming', 'is', 'awesome!']
print(list(filter(lambda arg: len(arg) < 8, x)))

['Python', 'is']


filter()需要一个迭代，调用lambda函数到每项上，并返回其中 lambda返回True 的那些项。上面的例子等价于：

In [5]:
def is_less_than_8_characters(item):
    return len(item) < 8

x = ['Python', 'programming', 'is', 'awesome!']
results = []

for item in x:
    if is_less_than_8_characters(item):
        results.append(item)

print(results)

['Python', 'is']


filter() 返回的是一个iterable，这意味着filter()函数不需要电脑又足够的内存来一次性持有iterable中的所有项。这种处理大数据集的时候非常重要。

map()函数和filter()函数类似，也是作用到一个iterable中的每一项，不过其输出是一个对原iterable中各项一对一的新的iterable。这个和filter函数略有不同。

In [6]:
x = ['Python', 'programming', 'is', 'awesome!']
print(list(map(lambda arg: arg.upper(), x)))

['PYTHON', 'PROGRAMMING', 'IS', 'AWESOME!']


map函数自动调用lambda函数到所有项，有效地代替了循环。上面的例子和下面代码功能一样：

In [7]:
results = []

x = ['Python', 'programming', 'is', 'awesome!']
for item in x:
    results.append(item.upper())

print(results)

['PYTHON', 'PROGRAMMING', 'IS', 'AWESOME!']


map()返回一个可迭代的对象，这使它有可能处理大的数据集(这些数据集太大而无法完全容纳在内存中)。

Python标准库中的另一个功能是reduce()。与filter()和map()一样，reduce()将函数应用于可迭代的元素。

同样，要应用的函数可以是使用def关键字或函数创建的标准Python lambda函数。

但是，reduce()不会返回新的可迭代对象。而是使用调用的函数将Iterable减少为单个值：

In [8]:
from functools import reduce
x = ['Python', 'programming', 'is', 'awesome!']
print(reduce(lambda val1, val2: val1 + val2, x))

Pythonprogrammingisawesome!


下面看看这些概念是如何扩展到PySpark API中以处理大量数据的。

set是标准Python中存在的另一种常见功能，在大数据处理中广泛有用。其与列表非常相似，除了它们没有任何顺序并且不能包含重复值。可以认为set类似于Python dict中的键。

pyspark 程序和平常的python程序没有太多区别，不过execution model和普通的python程序是很不同的，尤其是在集群上运行时。开始的时候还是仅将使用pyspark作为一个普通python程序调用的library较好。

那么什么是Spark？Apache Spark 由一些组件组成，直接描述它还是有些困难。核心部分，Spark 是一个 **generic engine** for 处理大数据。这里先了解下核心的Spark components 是如何处理大数据的。

Spark 是由 Scala 编写，在JVM上运行，所以自然地要使用PySpark首先需要安装Java，以及Spark，当然还有Python。由built-in components来处理流数据，机器学习，graph processing和通过SQL与数据交互。这里用python来交互的话，需要使用PySpark。可以认为其是Scala API的一个python包装。所以需要关注 [Spark Scala的文档](https://spark.apache.org/docs/latest/api/scala/index.html#package)，也要关注 [PySpark的文档](http://spark.apache.org/docs/latest/api/python/index.html)。很多时候还是需要查看 Scala 的文档。PySpark 和 Spark Scala-based API交互主要通过[Py4J 库](https://www.py4j.org/)完成。Py4J 不仅仅针对 Sparj，它是一个允许Python程序和JVM-based代码交互的库。PySPark是基于函数式范式的：

1. Spark’s native language, Scala, is functional-based.
2. Functional code is much easier to parallelize.

在Python环境下，可以认为PySpark 能进行并行处理而不需要 threading 或 multiprocessing 库。所有复杂的threads，processes和不同CPUs之间的communication和synchronization都交给Spark处理。

Spark 是一个 master-slave 地架构，master被称作 Driver，slaves称为workers。当运行一个Spark程序时，Spark Driver 创建一个context，这个context就是应用程序的入口，所有的操作（transformations和actions）都在worker nodes上执行，资源由Cluster Manager管理。

![](QQ截图20201027110638.png)

有不同的Cluster Manager 类型:

- Standalone – a simple cluster manager included with Spark that makes it easy to set up a cluster.
- Apache Mesos – Mesons is a Cluster manager that can also run Hadoop MapReduce and PySpark applications.
- Hadoop YARN – the resource manager in Hadoop 2. This is mostly used, cluster manager.
- Kubernetes – an open-source system for automating deployment, scaling, and management of containerized applications.

后面用到再细说。一开始的时候还是会以本地机器为主，这时候cluster manager就是local，虽然不少个真正的cluster manager，但是初学还是会用local for master()来在本地电脑运行Spark。

下图是PySpark的一个概述图。

![](QQ截图20201027105753.png)

- SparkConf 提供运行一个Spark应用的配置；
- SparkContext可以将自己的程序设置在Spark环境下，任何Spark程序的entry-point都是一个SparkContext对象，该对象让我们能连接到一个Spark 集群上并创建RDDs。如下图所示；

![](QQ截图20201027112322.png)

- SparkFiles允许我们处理系统或分布式系统上的文件，用sc.addFile可以上传自己的文件，使用SparkFiles.get可以获取worker上的路径；
- RDD是为了和PySpark 交互，需要建立的特定数据结构： [Resilient Distributed Datasets](https://spark.apache.org/docs/latest/rdd-programming-guide.html#resilient-distributed-datasets-rdds)(RDDs)，如果在一个集群上，RDDs 会隐藏自动地通过scheduler在多节点上所有转换和分布数据的复杂性。RDD是Spark上基本的abstraction，它代表了能被并行运算的immutable，partitioned collection of elements，如下图所示。
    - 有许多方法创建RDDs，最常见的方法是 PySpark的parallelize()函数。该函数将转换Python的数据结构到RDDs，使他们能分布式化。可以认为Spark在做multiprocessing的工作，只不过都封装在RDD的数据结构中了。
    - take()函数是查看RDD中内容的方法，但支队小子集。take函数拉取分布式系统上数据的子集到单个机器上。take函数对debug比较重要，因为在一个机器上检查所有数据是不太可能的。
    - 另一种常见的创建RDDs的方法是通过textFile()函数读取一个文件。更多细节后面用到再补充。
    - RDDs和其他数据结构一个重要区别在于：processing is delayed until the result is requested.即lazy evaluation式的运算。
    - 在同一个RDD上可以堆叠多个transformations。因为Spark 会维护一个transformations的有向无环图
    - 有许多方法从RDD request 结果。可以使用collect()函数显式地请求结果，并收集到一个集群节点上，也可以隐式地，比如count()函数。

![](QQ截图20201027113631.png)

- StorageLevel决定了RDD是在存储在内存中，还是存储在硬盘中，或者both
- DataFrames是一个分布式collection of rows with named columns
- Broadcast and Accumulator： Broadcast 变量能让程序员保持一个只读变量缓存到每个机器上而不是copy；Accumulators是对变量累计运算。这俩有点像mapreduce了。

下面先看看如何安装PySpark。开始的时候自然还是在local machine上试一试，不够PySpark的运行并不是太容易。因为它运行在JVM上，所以需要很多Java infrastructure。因为我现在是在windows下运行，所以还需要特别处理下，可以用一下Docker来快速使用，不过这里我还是暂时先根据教程直接安装了，以后在linux系统下再补充Docker使用的相关方法。

首先，需要安装Python，这个前面最开始的时候就说了，这里就不赘述了。

接下来安装Java。为了运行PySpark，需要安装Java 8及之后的版本。因为我做大数据处理还是主要为了地理空间处理，后面会用到geospark，而geospark稳定版的spark版本目前还是2.X，所以暂时还是先以Spark 2.X对应的版本为主。Java的话安装Java8即可，java8应该也是目前用的仍然比较多的版本，在[Oracle 网站](https://www.oracle.com/java/technologies/javase/javase-jdk8-downloads.html)下载安装包，然后点击安装即可，我这里都是默认的安装配置，所以就不多说了。安装之后需要配置下环境变量，可以参考：https://www.runoob.com/java/java-environment-setup.html：

``` PATH
<!-- 新建系统变量 -->
JAVA_HOME = C:\Program Files\Java\jdk1.8.0_271
<!-- 新建PATH -->
%JAVA_HOME%\bin
%JAVA_HOME%\jre\bin
```

然后测试下安装成功没有：

```SHELL
java -version
```

有相关信息就是安装成功了。

下面开始安装 Apache Spark。直接去[Spark网站](https://spark.apache.org/downloads.html)，我这里使用Spark 2.4.7 版本。

![](QQ截图20201027162102.png)

点击Download Spark后面给得链接即可进入下载页，然后点击推荐得镜像下载即可。

下载之后，解压，然后我这里把文件夹放到 C:\ProgramData 下，所以文件夹就是 C:\ProgramData\spark-2.4.7-bin-hadoop2.7 

接下来设置下环境变量：

``` PATH
<!-- 新建系统变量 -->
SPARK_HOME  = C:\ProgramData\spark-2.4.7-bin-hadoop2.7
HADOOP_HOME = C:\ProgramData\spark-2.4.7-bin-hadoop2.7
<!-- 新建PATH -->
%SPARK_HOME%\bin
```

为了在windows下运行还需要一个工具 [wintuils](https://github.com/steveloughran/winutils)，注意要下载自己得hadoop对应得版本。我这里hadoop是2.7。在winutils github页面进入 hadoop-2.7.1/bin，找到winutils.exe，download即可，然后将其copy到%SPARK_HOME%\bin 文件夹下。

然后就可以看看PySpark是不是能运行了，打开命令行，输入pyspark，应该能看到类似得结果：

![](QQ截图20201027164236.png)

Spark-shell 也创建了一个 Spark context web UI 并且默认地, 可以通过 http://localhost:4040/ 访问，参考：[Spark Web UI – Understanding Spark Execution](https://sparkbyexamples.com/spark/spark-web-ui-understanding/)。

应该能看到类似界面：

![](QQ截图20201027164600.png)

Spark 提供了一系列的 Web UI（Jobs，Stages，Tasks，Storage，Environment，Executors，和SQL）来检测Spark应用的状态，Spark集群资源使用情况，和Spark配置等。在Spark Web UI中，可以看到operations是如何被执行的。

另外，Spark History servers 能记录通过 spark-submit，sparl-shell提交的所有Spark 应用。开始前，需要设置spark-defaults.conf：

```Config
spark.eventLog.enabled true
spark.history.fs.logDirectory file:///c:/logs/path
```

然后在windows下，运行下面命令：

```Shell
SPARK_HOME/bin/spark-class.cmd org.apache.spark.deploy.history.HistoryServer
```

这里我暂时没找到spark-defaults.conf位置，所以先不弄了，后面用到再来处理。

下面是Jupyter lab里面运行，在jupyter中运行pyspark，有不同的设置方法，参考：[How to install PySpark and Jupyter Notebook in 3 Minutes](https://www.sicara.ai/blog/2017-05-02-get-started-pyspark-jupyter-notebook-3-minutes)，有两种方法：

- Configure PySpark driver to use Jupyter Notebook: running pyspark will automatically open a Jupyter Notebook
- Load a regular Jupyter Notebook and load PySpark using findSpark package

因为我是想直接在现有的jupyter 下运行 pyspark的，所以就暂时采用第二种方法，虽然其稍微麻烦一些，但是是适合各种IDE的。首先安装 findspark，在本repo首页README文件里已有提及：

```Shell
conda install -c conda-forge findspark
```

然后在import pyspark之前，首先import findspark 并 findspark.init() 。如果是刚刚安装好 spark 和 findspark，还是要重启下jupyter。

In [2]:
import findspark
findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 1000

def inside(p):     
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()

3.312


更多并行计算实际应用参考7-parallel-programming/parallel-tutorial-master 文件夹下的内容。